In [125]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from Kildekode._12_IIRFilt import *

%matplotlib ipympl

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# IIR Filtre Introduksjon

* **Emne IELEA2302 - Signalbehandling**
* **Uke 12, 2021**
* **Relevant Pensum:**
    * Kapittel 10.1-10.2 i læreboka DSP First
* **Underviser: Kai Erik Hoff**

# Oppsummering FIR filtre

* **FIR** - **F**inite **I**mpulse **R**esponse
* Fellesbetegnelse for filter som kalkulerer utgangsveriden $y[n]$ *kun* basert på en lineær kombinasjon av verdiene til inngangssignalet $x[n]$ ved forskjellige sampletidspunkt.
$$y[n] = \sum_{k=0}^{M} b_k \cdot x[n-k]$$
* Impulsresponsen $h[n]$ til et FIR filter gjengir filterkoeffisientene $b_k$
$$h[n] = \sum_{k=0}^{M} b_k \cdot \delta[n-k]$$
* Filtrereing av et signal $x[n]$ tilsvarer å utføre konvolusjon mellom signalet og impulsresponsen $h[n]$.
$$y[n] = x[n]*h[n]$$

# Oppsummering Frekvensrespons

* ***Frekvensresponsen*** til et filter med impulsrespons $h[n]$ er impulsresponsens fourietransformerte:
$$H\left(e^{j\hat{\omega}} \right) = \text{DTFT}(h[n]) = \sum_{n=0}^{\infty} h[n]\cdot e^{-j\hat{\omega}\cdot n}$$
* Gir forholdet mellom inngangssignal og utgangssignal til et filter i frekvensplanet.
$$Y\left(e^{j\hat{\omega}} \right) = X\left(e^{j\hat{\omega}} \right)\cdot H\left(e^{j\hat{\omega}} \right) \leftrightarrow H\left(e^{j\hat{\omega}} \right) = \frac{Y\left(e^{j\hat{\omega}} \right)}{X\left(e^{j\hat{\omega}} \right)}$$
* Forteller hvordan filteret vil påvirke en hvilken som helst sinussekvens:
\begin{align}
x[n] &=  A\cdot \cos(\hat{\omega}_0 \cdot n + \phi)\\
& \downarrow \\
y[n] &= \left| H\left(e^{j\hat{\omega}} \right)\right| \cdot A\cdot \cos\left(\hat{\omega}_0 \cdot n + \phi + \angle H\left(e^{j\hat{\omega}} \right)\right)
\end{align}

# IIR filtre

* **IIR** - **I**nfinite **I**mpulse **R**esponse
* Fellesbetegnelse for filter som kalkulerer utgangsveriden $y[n]$ basert på en lineær kombinasjon av verdiene til inngangssignalet ved foregående sampletidspunkt $x[n-k], \ \ k\in \{0, 1, 2, \ldots\}$ ***og*** tidligere kalkulerte verdier til utgangssignalet $y[n-k], \ \ k \in \{1, 2, \ldots\}$.


* **Differanseligning:**
$$\sum_{k=0}^{N} a_k \cdot y[n-k] = \sum_{k=0}^{M} b_k \cdot x[n-k]$$
* **Filteralgoritme:**
$$y[n] = \frac{1}{a_0}\cdot \left(\sum_{k=0}^{M} b_k \cdot x[n-k] - \sum_{k=1}^{N}a_k\cdot  y[n-k] \right)$$

## Mellomregning differanseligning $\rightarrow$ filteralgoritme
\begin{align}
\sum_{k=0}^{N} a_k \cdot y[n-k] &= \sum_{k=0}^{M} b_k \cdot x[n-k]\\
a_0\cdot y[n]+\sum_{k=1}^{N} a_k \cdot y[n-k] &= \sum_{k=0}^{M} b_k \cdot x[n-k]\\
a_0\cdot y[n] &= \sum_{k=0}^{M} b_k \cdot x[n-k]-\sum_{k=1}^{N} a_k \cdot y[n-k]\\
y[n] &=\frac{1}{a_0}\cdot \left(\sum_{k=0}^{M} b_k \cdot x[n-k] - \sum_{k=1}^{N}a_k\cdot  y[n-k] \right)
\end{align}

# Rekursiv filterstruktur

* IIR filtre omtales ofte som *rekursive* filtre ettersom utgangssignalet tilbakekobles som input.
<img src="Figurer/12_IIR%20Filtre/Fig0_Recursion.png" style="width: 35%; margin-left: 400px" />

* Impulsresponsen $h[n]$ for et slikt filter vil ha uendelig varighet.
* Impulsrespons og filterkoeffisienter er ikke lengre synonyme.
$$b_k \neq h[n]\big{|}_{n=k}$$

* Ettersom impulsresponsen har uendelig varighet, er det ikke hensiktsmessig utføre filtrering ved å regne ut konvolusjon numerisk. Da bruker man heller filteralgoritmen, og finner utgangssamplene i kronologisk rekkefølge.

# Blokkskjema av filteralgoritme

* Såkalt **Direkte Form I** Implementasjon:
<img src="Figurer/12_IIR%20Filtre/Fig1_DF1.png" style="width: 60%; margin-left: 200px" />

* Det er *nesten alltid* ønskelig å ***normalisere*** filterkoeffisientene slik at $a_0 = 1$.

## Regneeksempel 1:


<img src="Figurer/12_IIR%20Filtre/Fig2_DF1_ex.png" style="width: 35%; margin-left: 100px" />

1. Finn *filterkoeffisientene* til filteret i figuren.
2. Finn et uttrykk for filterets impulsrespons $h[n]$.

# Generell filtrering av signal i Python

* Filtrering av signal med både IIR og FIR filter kan enkelt utføres med bruk av funksjonen [`lfilter()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html) i modulen [`scipy.signal`](https://docs.scipy.org/doc/scipy/reference/tutorial/signal.html).
* Argument:
    * **`b`**: array med filterkoeffisienter $b_k$. 
    * **`a`**: array med filterkoeffisienter $a_k$.
    * **`x`**: array med hele inngangssignalet $x[n]$.
* Returverdi:
    * **`y`**: array med utgangssignal. 
        * *NB! I motsetning til `np.convolve()` har `x` og `y` samme lengde.*

## Kodeeksempel 1:
* Bruk funksjonen `lfilter()` til å regne ut impulsresponsen for filteret i *regneeksempel 1* for sampleintervallet $-4\leq n <60$, og vis det resulterende utgangssignalet i et stolpediagram.


# Stegrespons *(sprangrespons)*

* Utgangssignalet $y[n]$ for et LTI-system når inngangssignalet er en *enhetsstegfunksjon* $x[n] = u[n]$ er kjent som filterets stegrespons $s[n]$
* Synliggjør filterets "responstid". 
* IIR-filtre vil som regel konvergere mot en stasjonær verdi.

<img src="Figurer/12_IIR%20Filtre/Fig3_StepResp.png" style="width: 80%; margin-left: 100px" />

# Stegrespons og impulsrespons

<br>
<div style="width: 100%;">
    <div style="float: left; width: 50%">
    <ul>
      <li><b>Impulsrespons til stegrespons</b></li>
        <br>
        \begin{align}
        s[n] &= h[n]*u[n]\\
        &= \sum_{k=0}^{\infty}h[k]\cdot u[n-k]\\
        &= \sum_{k=0}^{n-1}h[k]
        \end{align}
    </ul>  
    </div>
    <div style="float: right; width: 50%">
    <ul>
      <li><b>Stegrespons til impulsrespons</b></li>
        <br>
        \begin{align}
        \delta[n] &= u[n] - u[n-1]\\
        & \downarrow\\
        h[n] &= s[n]-s[n-1]
        \end{align}
    </ul>  
    </div>
</div> 




## Regneeksempel 2:

* Finn stegresponsen til filteret i eksempel 1.
* Justér filterkoeffisientene slik at stegresponsen konvergerer mot 1: $$\lim_{n\rightarrow \infty} s[n] = 1$$

# Frekvensrespons førsteordens IIR-filter

* Vi kan fortsatt finne frekvensresponsen til ved å fouriertransformere impulsresponsen $h[n]$.
\begin{align}
y[n] &= b_0 \cdot x[n] - a_1 \cdot y[n-1]\\
&\downarrow \\
h[n] &= b_0\cdot u[n] \cdot (-a_1)^n \\
&\downarrow \\
H\left(e^{j\hat{\omega}}\right) &= \frac{b_0}{1+a_1\cdot e^{-j\hat{\omega}}}, \ \ \ |a_1| < 1
\end{align}
* DTFT av geometrisk rekke gitt i tabell over vanlige fouriertransformasjonspar.

* Hva skjer dersom $|a_1| \geq 1$?

In [134]:
# Frekvensresponsen til filteret i eksempel 3
FreqRespDemo([0.1], [1, -0.9], fig_num=3);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## Amplituderespons IIR og FIR filter

In [128]:
# Førsteordens IIR-filter
w, Hw1 = sig.freqz([0.1], [1, -0.9])
# 21-punkts middelverdifilter 
w, Hw2 = sig.freqz(np.ones(21)/21)

# Vis amplituderesponsen til de to filtrene i samme figur
plt.close(2); plt.figure(2)
plt.plot(w, np.abs(Hw1), label=r'IIR-filter')
plt.plot(w, np.abs(Hw2), label=r'Middelverdifilter')
plt.grid(True)
plt.xticks(np.linspace(0, 1, 6)*pi, [str(round(i,2))+r'$\pi$' for i in np.linspace(0, 1, 6)])
plt.xlabel(r'Digital Frekvens $\hat{\omega}$')
plt.ylabel(r'$\left| H\left(e^{j\hat{\omega}}\right)\right|$')
plt.xlim([0, pi]); plt.ylim(ymin=0)
plt.title(r'Amplituderespons')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Regneeksempel 3: 
* Gitt høypassfilteret
$$y[n] = x[n]-x[n-1]-0.8\cdot y[n-1]$$
Finn et uttrykk for frekvensresponsen, og korriger deretter filterkoeffisientene slik at $\text{max}\left(\left|H\left(e^{j\hat{\omega}}\right)\right|\right) = 1$.

In [135]:
# Frekvensresponsen til filteret i eksempel 3
FreqRespDemo([0.1, -0.1], [1, 0.8], fig_num=3);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

# Refleksjoner

* Ved å lage et *rekursivt* filter, kan selv enkle førsteordens filtre produsere en langvarig impulsrespons.
* Et IIR-filter med relativt lav filterorden kan, dersom vi velger filterkoeffisienter med omhu, oppfylle de samme kravene for amplitduerespons som et FIR filter med ***mye*** høyere filterorden.
* Ulempen er at IIR filtre ***ikke*** har lineær faserespons i passbåndet. 
    * Konsekvensen av dette er *tidsforskyvningen* for hver av frekvenskomponentene til det filtrerte signalet gitt i antall sampler ikke vil være konstant.
    * Dette har potensiale til å forårsake uønsket distorsjon av f.eks. et lydsignal.

* For å kunne analysere IIR-filter med filterorden $>1$, trenger vi et nytt verktøy: **Z-transformen**
    * Z-transformen kan regnes som Laplacetransformens diskréte utgave.
    * Denne venter vi med til etter påske

# God Påske! 